In [478]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {} 
        
    def inc(self, numOccur):
        self.count += numOccur
        
    def disp(self, ind=1):
        print '  '*ind, self.name, ' ', self.count
        for child in self.children.values():
            child.disp(ind+1)

In [479]:
def loadSimDat():
    simDat = [['r','z','h','j','p'],
              ['z','y','x','w','v','u','t','s'],
              ['z'],
              ['r','x','n','o','s'],
              ['y','r','x','z','q','t','p'],
              ['y','z','x','e','q','s','t','m']
              ]
    return simDat
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [480]:
loadSimDat()

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [481]:
InitSet = createInitSet(loadSimDat())

In [482]:
InitSet

{frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'h', 'j', 'p', 'r', 'z'}): 1}

In [483]:
rootNode = treeNode('A',8,None)
rootNode.disp()

   A   8


In [484]:
rootNode.children['B'] = treeNode('B',7,None)
rootNode.disp()

   A   8
     B   7


In [485]:
def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    headerTable = {}
    #go over dataSet twice
    # 計算每個Transaction中item的數量，存於dict中
    for trans in dataSet:#first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    #remove items not meeting minSup
    for k in headerTable.keys():  
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print 'freqItemSet: ',freqItemSet
    #if no items meet min support -->get out
    if len(freqItemSet) == 0:
        #print('no items meet min support')
        return None, None
    #reformat headerTable to use Node link 
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] 
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree
    #retTree.disp()
    #print(dataSet)
    # key:tranSet value:count
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        # 一筆 Transaction 
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        #print(localD)
        if len(localD) > 0:
            # [('y', 3), ('x', 4), ('s', 3), ('z', 5), ('t', 3)] ->[('z', 5), ('x', 4), ('y', 3), ('s', 3), ('t', 3)]
            # ['z', 'x', 'y', 's', 't']
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)#populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table

In [486]:
def updateTree(items, inTree, headerTable, count):
    #print('updateTree')
    #print(items)
    #print(inTree)
    #inTree.disp()
    # 每一筆 Transaction 的每一個item -> items[0]
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        #update header table
        if headerTable[items[0]][1] == None:  
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    #call updateTree() with remaining ordered items
    if len(items) > 1:
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

In [487]:
def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [488]:
myFPtree,myHeaderTab = createTree(InitSet,3)

In [489]:
def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)
    
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [490]:
myHeaderTab

{'r': [3, <__main__.treeNode instance at 0x149a95a70>],
 's': [3, <__main__.treeNode instance at 0x1449d7488>],
 't': [3, <__main__.treeNode instance at 0x1449d7440>],
 'x': [4, <__main__.treeNode instance at 0x149a9e998>],
 'y': [3, <__main__.treeNode instance at 0x144a07440>],
 'z': [5, <__main__.treeNode instance at 0x149a9eea8>]}

In [491]:
myFPtree.disp()

   Null Set   1
     x   1
       s   1
         r   1
     z   5
       x   3
         y   3
           s   2
             t   2
           r   1
             t   1
       r   1


In [492]:
findPrefixPath('r',myHeaderTab['r'][1])

{frozenset({'s', 'x'}): 1, frozenset({'z'}): 1, frozenset({'x', 'y', 'z'}): 1}

In [493]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1])]#(sort header table)
    #print(headerTable.items())
    #print(bigL)
    for basePat in bigL:  #start from bottom of header table
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        #print 'finalFrequent Item: {0}'.format(newFreqSet)
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
       # print 'condPattBases :{0}---{1}'.format(basePat,condPattBases)
        #2. construct cond FP-tree from cond. pattern base
        myCondTree, myHead = createTree(condPattBases, minSup)
        #print 'myCondTree: {0}'.format(myCondTree)
        #print 'head from conditional tree: {0}'.format(myHead)
        if myHead != None: #3. mine cond. FP-tree
            #print 'conditional tree for: ',newFreqSet
            #myCondTree.disp(1)            
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [494]:
freqItems=[]

In [495]:
mineTree(myFPtree,myHeaderTab, 3,set([]),freqItems)

In [496]:
freqItems

[{'t'},
 {'t', 'y'},
 {'t', 'x'},
 {'t', 'x', 'y'},
 {'t', 'z'},
 {'t', 'y', 'z'},
 {'t', 'x', 'z'},
 {'t', 'x', 'y', 'z'},
 {'s'},
 {'s', 'x'},
 {'y'},
 {'x', 'y'},
 {'y', 'z'},
 {'x', 'y', 'z'},
 {'r'},
 {'x'},
 {'x', 'z'},
 {'z'}]

In [273]:
datapath = '/Users/wy/Desktop/kosarak.dat'

In [274]:
parsedDat = [line.split() for line in open(datapath).readlines()]

In [275]:
initSet = createInitSet(parsedDat)

In [276]:
myFPtree,myHeaderTab = createTree(initSet,100000)

In [277]:
myList = []
mineTree(myFPtree,myHeaderTab,100000,set([]),myList)

conditional tree for:  set(['1'])
conditional tree for:  set(['3'])
conditional tree for:  set(['11', '3'])
conditional tree for:  set(['11'])


In [278]:
myList

[{'1'},
 {'1', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'11'},
 {'11', '6'},
 {'6'}]

In [497]:
datapath = '/Users/wy/Desktop/retail.dat'
dataList=[]
with open (datapath,'r') as f:
    lines = f.readlines()
for index, line in enumerate(lines):
    line=line.strip()
    listLine = line.split(' ')
    dataList.append(listLine)

In [533]:
tinitSet = createInitSet(dataList)
tmyFPtree,tmyHeaderTab = createTree(tinitSet,8800)
tmyList = []
mineTree(tmyFPtree,tmyHeaderTab,8800,set([]),tmyList)

In [534]:
tmyList

[{'32'},
 {'41'},
 {'41', '48'},
 {'39', '41'},
 {'38'},
 {'38', '39'},
 {'48'},
 {'39', '48'},
 {'39'}]

In [535]:
len(tmyList)

9

In [544]:
tmyFPtree.children['38'].children

{'32': <__main__.treeNode instance at 0x147d3b098>,
 '41': <__main__.treeNode instance at 0x147d3b248>}

In [547]:
tmyFPtree.disp()

   Null Set   1
     32   2784
     39   47860
       32   1795
       38   3977
         32   415
         41   1020
           32   170
       48   28184
         32   2910
         38   5976
           32   783
           41   1971
             32   447
         41   5278
           32   1190
       41   2834
         32   524
     38   3008
       32   447
       41   451
         32   91
     48   12556
       32   1801
       38   1786
         32   314
         41   380
           32   92
       41   1257
         32   325
     41   1378
       32   317


In [538]:
tmyHeaderTab

{'32': [14405, <__main__.treeNode instance at 0x147d30cb0>],
 '38': [14747, <__main__.treeNode instance at 0x147d30d40>],
 '39': [47860, <__main__.treeNode instance at 0x147d30b00>],
 '41': [14569, <__main__.treeNode instance at 0x147d30c68>],
 '48': [40740, <__main__.treeNode instance at 0x147d30b48>]}

In [ ]:
[[48], [38], [32], [39], [41], [48, 41], [38, 39], [48, 39], [41, 39]]

In [548]:
float(2834)/float(47860)

0.05921437526117844